In [1]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks.callbacks import LearningRateScheduler



Using TensorFlow backend.


In [2]:
IM_SIZE = (299,299)

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,  
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   fill_mode='nearest')

train_batches = train_datagen.flow_from_directory('data/19_class/train',
                                                  classes=['0Parus', '1Passe', '2Lusci',
                                                   '3Phoen', '4Erith', '5Phoen',
                                                   '6Sitta', '7Alaud', '8Phyll',
                                                   '9Turdu', '10Phyll', '11Fring',
                                                   '12Sturn', '13Ember', '14Colum',
                                                   '15Trogl', '16Cardu', '17Chlor', '18Turdu'],
                                                  target_size=IM_SIZE,
                                                  class_mode='categorical', shuffle=True,
                                                  batch_size=16)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_batches = valid_datagen.flow_from_directory('data/19_class/val',
                                                  classes=['0Parus', '1Passe', '2Lusci',
                                                   '3Phoen', '4Erith', '5Phoen',
                                                   '6Sitta', '7Alaud', '8Phyll',
                                                   '9Turdu', '10Phyll', '11Fring',
                                                   '12Sturn', '13Ember', '14Colum',
                                                   '15Trogl', '16Cardu', '17Chlor', '18Turdu'],
                                                  target_size=IM_SIZE,
                                                  class_mode='categorical', shuffle=False,
                                                  batch_size=16)

# show class indices
print('****************')
for cls, idx in train_batches.class_indices.items():
    print('Class nr ',idx,' -> ', cls)
print('****************')


Found 118716 images belonging to 19 classes.
Found 13972 images belonging to 19 classes.
****************
Class nr  0  ->  0Parus
Class nr  1  ->  1Passe
Class nr  2  ->  2Lusci
Class nr  3  ->  3Phoen
Class nr  4  ->  4Erith
Class nr  5  ->  5Phoen
Class nr  6  ->  6Sitta
Class nr  7  ->  7Alaud
Class nr  8  ->  8Phyll
Class nr  9  ->  9Turdu
Class nr  10  ->  10Phyll
Class nr  11  ->  11Fring
Class nr  12  ->  12Sturn
Class nr  13  ->  13Ember
Class nr  14  ->  14Colum
Class nr  15  ->  15Trogl
Class nr  16  ->  16Cardu
Class nr  17  ->  17Chlor
Class nr  18  ->  18Turdu
****************


In [3]:
net = Xception(include_top=False,
                        weights='imagenet',
                          input_tensor=None,
                        input_shape=(299,299,3))
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(19, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:20]:
    layer.trainable = False
for layer in net_final.layers[20:]:
    layer.trainable = True
net_final.compile(optimizer=Adam(lr=5e-4),
                  loss='categorical_crossentropy', metrics=['accuracy'])
print(net_final.summary())



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [ ]:
import numpy as np

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_batches.classes), 
                train_batches.classes)
class_weights
# train the model
net_final.fit_generator(train_batches,
                        validation_data = valid_batches,
                        epochs = 20,
                        steps_per_epoch= 1000,
                        class_weight=class_weights)


Epoch 1/20
1000/1000 [==============================] - 14204s 14s/step - loss: 1.8117 - accuracy: 0.4761 - val_loss: 1.3943 - val_accuracy: 0.6328
Epoch 2/20
1000/1000 [==============================] - 14167s 14s/step - loss: 1.2834 - accuracy: 0.6349 - val_loss: 1.0905 - val_accuracy: 0.7100
Epoch 3/20
1000/1000 [==============================] - 14283s 14s/step - loss: 1.1393 - accuracy: 0.6754 - val_loss: 1.0444 - val_accuracy: 0.6914
Epoch 4/20
1000/1000 [==============================] - 14225s 14s/step - loss: 1.0489 - accuracy: 0.6985 - val_loss: 0.9837 - val_accuracy: 0.6924
Epoch 5/20
1000/1000 [==============================] - 13979s 14s/step - loss: 1.0009 - accuracy: 0.7089 - val_loss: 0.9404 - val_accuracy: 0.7297
Epoch 6/20
 727/1000 [====================>.........] - ETA: 52:19 - loss: 0.9483 - accuracy: 0.7295

In [ ]:
# save trained weights
net_final.save('models/AM_xception_19classes.h5')

In [ ]:
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train the model
net_final.fit_generator(train_batches,
                        validation_data = valid_batches,
                        epochs = 20)


In [ ]:
net_final.compile(optimizer=Adam(lr=3e-6),
                  loss='categorical_crossentropy', metrics=['accuracy'])
# train the model
net_final.fit_generator(train_batches,
                        steps_per_epoch = 8*20,
                        validation_data = valid_batches,
                        epochs = 20)
